# PREPARE DATA

In this notebook, we treat raw data from the original Excel files to be used in subsequent steps. The treatment includes:
- Filtering: city, relevant columns.
- Removal: NaNs and entries with wrong types.
- Fixing: latitudes an longitudes have both entries with different orders of magnitude.

## Notebook setup

In [1]:
import numpy as np

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import pickle

In [2]:
pd.set_option('display.max_columns', None)

In [ ]:
# TODO: solve inconsistencies found in temporal data
#BASE_COLS = ['DATAOCORRENCIA', 'HORAOCORRENCIA', 'PERIDOOCORRENCIA',]
# QUICKFIX: use first day of month as incident date

# time of the day
BASE_COLS = ['PERIDOOCORRENCIA']

# Neighborhoods dataframe
COLS_HOOD = BASE_COLS + ['BAIRRO']

# Localizations dataframe
COLS_LOC = BASE_COLS + ['LATITUDE', 'LONGITUDE']

In [ ]:
# run the notebook for each of these valid values
INCIDENT = 'furto_celular' #'roubo_celular' #'furto_veiculo' #'roubo_veiculo' #

# original data (not included in repository)
DIR_DATA_CRIMES = "../data/crimes"
PATH_CRIMES = f"{DIR_DATA_CRIMES}/{INCIDENT}"

# TODO:
#OUTPUT_FILE = "data/crimes/{INCIDENT}.pkl"

we have different time periods for each type of crime,

In [5]:
if INCIDENT in ['furto_celular', 'roubo_veiculo']:
    periods = [f'2023_{i:02d}' for i in range(1, 13)]
if INCIDENT in ['furto_veiculo', 'roubo_celular']:
    periods = [f'2023_{i:02d}' for i in range(1, 13)] + ['2024_01']
periods

['2023_01',
 '2023_02',
 '2023_03',
 '2023_04',
 '2023_05',
 '2023_06',
 '2023_07',
 '2023_08',
 '2023_09',
 '2023_10',
 '2023_11',
 '2023_12']

In [6]:
def fixlatlon(n) -> float:
    """Transforms a real number to be in the (-100, 100) range

    Example:
    >>> fixlatlon(-123456.789)
    np.float64(-12.345678900000001)"""

    tenpow = np.ceil(np.log10(np.abs(float(n)))) - 2
    return n / (10**tenpow)

## Treat and join dataframes

In [7]:
# just for reference,
dfx = pd.read_excel(f"{PATH_CRIMES}/2023_12.xlsx")

In [8]:
dfx.columns

Index(['ANO_BO', 'NUM_BO', 'NUMERO_BOLETIM', 'BO_INICIADO', 'BO_EMITIDO',
       'DATAOCORRENCIA', 'HORAOCORRENCIA', 'PERIDOOCORRENCIA',
       'DATACOMUNICACAO', 'DATAELABORACAO', 'BO_AUTORIA', 'FLAGRANTE',
       'NUMERO_BOLETIM_PRINCIPAL', 'LOGRADOURO', 'NUMERO', 'BAIRRO', 'CIDADE',
       'UF', 'LATITUDE', 'LONGITUDE', 'DESCRICAOLOCAL', 'EXAME', 'SOLUCAO',
       'DELEGACIA_NOME', 'DELEGACIA_CIRCUNSCRICAO', 'ESPECIE', 'RUBRICA',
       'DESDOBRAMENTO', 'STATUS', 'TIPOPESSOA', 'VITIMAFATAL', 'NATURALIDADE',
       'NACIONALIDADE', 'SEXO', 'DATANASCIMENTO', 'IDADE', 'ESTADOCIVIL',
       'PROFISSAO', 'GRAUINSTRUCAO', 'CORCUTIS', 'NATUREZAVINCULADA',
       'TIPOVINCULO', 'RELACIONAMENTO', 'PARENTESCO', 'PLACA_VEICULO',
       'UF_VEICULO', 'CIDADE_VEICULO', 'DESCR_COR_VEICULO',
       'DESCR_MARCA_VEICULO', 'ANO_FABRICACAO', 'ANO_MODELO',
       'DESCR_TIPO_VEICULO', 'QUANT_CELULAR', 'MARCA_CELULAR'],
      dtype='object')

In [9]:
df_hood = None
df_loc = None
for period in periods:
    df = pd.read_excel(f"{PATH_CRIMES}/{period}.xlsx")
    # filter capital
    df = df[df["CIDADE"] == "S.PAULO"]

    # standarize neighborhood
    df["BAIRRO"] = df["BAIRRO"].str.upper()

    ## TODO: solve inconsistencies found in temporal data
    #df['DATAOCORRENCIA'] = pd.to_datetime(df['DATAOCORRENCIA'], format='%d/%m/%Y')
    #if DEBUG_MODE:
    #    print(
    #        f"{period}\t{df['DATAOCORRENCIA'].min()}\t{df['DATAOCORRENCIA'].max()}"
    #    )
    # QUICKFIX: use first day of month as incident date
    year, month = [int(s) for s in period.split(sep='_')]
    df["period"] = period
    df["year"] = year
    df["month"] = month
    df["date"] = pd.Timestamp(year=year, month=month, day=1)
    COLS_DT = ["period", "year", "month", "date"]

    df_hood = pd.concat([df_hood, df[COLS_HOOD + COLS_DT]])
    df_loc = pd.concat([df_loc, df[COLS_LOC + COLS_DT]])

    print(f"Period {period} done")

Period 2023_01 done
Period 2023_02 done
Period 2023_03 done
Period 2023_04 done
Period 2023_05 done
Period 2023_06 done
Period 2023_07 done
Period 2023_08 done
Period 2023_09 done
Period 2023_10 done
Period 2023_11 done
Period 2023_12 done


In [10]:
df_hood["BAIRRO"].value_counts()

BAIRRO
BELA VISTA               5488
BOM RETIRO               4467
PINHEIROS                3917
REPUBLICA                3892
LIBERDADE                3350
                         ... 
CIDADE D'ABRIL              1
JARDIM DONÁRIA              1
SHOPPING CENTER NORTE       1
VILA LISBOA                 1
JARDIM VARGINHA             1
Name: count, Length: 1442, dtype: int64

Remove NaNs,

In [11]:
print(df_hood.shape, df_loc.shape)
df_hood = df_hood[df_hood["BAIRRO"].notna()]
df_loc = df_loc[(df_loc["LONGITUDE"].notna())]
df_loc = df_loc[(df_loc["LATITUDE"].notna())]
print(df_hood.shape, df_loc.shape)

(123673, 6) (123673, 7)
(123449, 6) (94147, 7)


In [12]:
df_hood.groupby("date")["period"].count()

date
2023-01-01     4357
2023-02-01    10244
2023-03-01    11811
2023-04-01     9597
2023-05-01    11168
2023-06-01    11373
2023-07-01    10340
2023-08-01    11074
2023-09-01    12115
2023-10-01    10904
2023-11-01    10390
2023-12-01    10076
Name: period, dtype: int64

In [13]:
df_loc.groupby("date")["period"].count()

date
2023-01-01    3339
2023-02-01    7985
2023-03-01    8950
2023-04-01    7319
2023-05-01    8455
2023-06-01    8650
2023-07-01    7706
2023-08-01    8170
2023-09-01    9553
2023-10-01    8336
2023-11-01    7954
2023-12-01    7730
Name: period, dtype: int64

## Problem with `df_hood`: too much granularity

Municipio -> Prefeituras Regionais -> Distritos -> Bairros

In [14]:
df_hood["BAIRRO"].value_counts()

BAIRRO
BELA VISTA               5488
BOM RETIRO               4467
PINHEIROS                3917
REPUBLICA                3892
LIBERDADE                3350
                         ... 
CIDADE D'ABRIL              1
JARDIM DONÁRIA              1
SHOPPING CENTER NORTE       1
VILA LISBOA                 1
JARDIM VARGINHA             1
Name: count, Length: 1442, dtype: int64

Solutions:
- Map "BAIRRO" column to new "DISTRITO" column (hard).
- Use `df_loc` to intersect (lat, lon) with available district areas.

# Continue with `df_loc`

Fix types in location columns,

In [15]:
df_loc.dtypes

PERIDOOCORRENCIA            object
LATITUDE                   float64
LONGITUDE                   object
period                      object
year                         int64
month                        int64
date                datetime64[us]
dtype: object

In [16]:
df_loc["LATITUDE"].map(type).value_counts()

LATITUDE
<class 'float'>    94147
Name: count, dtype: int64

In [17]:
df_loc["LONGITUDE"].map(type).value_counts()

LONGITUDE
<class 'float'>    86162
<class 'int'>       7985
Name: count, dtype: int64

safe way: keep only floats,

TODO: maybe include ints

In [18]:
print(df_loc.shape)
df_loc = df_loc[df_loc["LATITUDE"].map(type) == float]
print(df_loc.shape)
df_loc = df_loc[df_loc["LONGITUDE"].map(type) == float]
print(df_loc.shape)

(94147, 7)
(94147, 7)
(86162, 7)


In [19]:
df_loc["LATITUDE"] = df_loc["LATITUDE"].astype(float)
df_loc["LONGITUDE"] = df_loc["LONGITUDE"].astype(float)

In [20]:
df_loc[["LONGITUDE", "LATITUDE"]].describe()

,LONGITUDE,LATITUDE
count,8.616200e+04,8.616200e+04
mean,-2.075460e+14,-1.123392e+14
std,2.296057e+14,1.169267e+14
min,-4.682100e+14,-2.388480e+14
25%,-4.663525e+14,-2.355036e+14
50%,-4.663513e+11,-2.352885e+13
75%,-4.666100e+08,-2.356540e+08
max,-4.662900e+04,-2.361200e+04


the orders of magnitudes of entries are absolutely absurd! solution:

## HARDFIX

123456789012345 -> 12.345...

In [21]:
fixlatlon(-123456.789)

np.float64(-12.345678900000001)

In [22]:
df_loc["LATITUDE"] = df_loc["LATITUDE"].apply(fixlatlon)
df_loc["LONGITUDE"] = df_loc["LONGITUDE"].apply(fixlatlon)

In [23]:
df_loc[["LATITUDE", "LONGITUDE"]].describe()

,LATITUDE,LONGITUDE
count,86162.000000,86162.000000
mean,-23.566058,-46.633211
std,0.061328,0.077192
min,-23.894869,-47.894961
25%,-23.588572,-46.680125
50%,-23.554231,-46.643002
75%,-23.533908,-46.616046
max,-20.719618,-46.365551


Remove absolutely extreme values of latitude and longitude (they still happen!),

In [24]:
# TODO: is there a way to guarantee that no fair data is lost?
print(df_loc.shape)
df_loc = df_loc[
        (df_loc["LATITUDE"] > df_loc["LATITUDE"].quantile(.0001)) & \
        (df_loc["LATITUDE"] < df_loc["LATITUDE"].quantile(.9999)) & \
        (df_loc["LONGITUDE"] > df_loc["LONGITUDE"].quantile(.0001)) & \
        (df_loc["LONGITUDE"] < df_loc["LONGITUDE"].quantile(.9999))
    ]
print(df_loc.shape)

(86162, 7)
(86127, 7)


Add column for the type of crime,

In [25]:
df_loc["crime"] = INCIDENT

Add geometry column,

In [26]:
df_loc["geometry"] = df_loc.apply(lambda x: Point(x["LONGITUDE"], x["LATITUDE"]), axis=1)

Create GeoDataFrame,

In [27]:
gdf_loc = gpd.GeoDataFrame(df_loc, geometry="geometry")

Save GeoDataFrame,

In [28]:
with open(f"{INCIDENT}.pkl", 'wb') as file:
    pickle.dump(gdf_loc, file)
print(f"{INCIDENT}.pkl saved")

furto_celular.pkl saved
